In [ ]:
# !gdown --id 1LukOUfVNeps1Jj7Z27JbkmrO90jwBgie
# !pip install kora
# from kora import drive
# drive.download_folder('1LukOUfVNeps1Jj7Z27JbkmrO90jwBgie')

In [4]:
import shutil
shutil.unpack_archive('mri.zip')

In [5]:
# !ls /content/img_align_celeba

### Load Libraries

In [6]:
# !pip install scipy==1.1.0

In [7]:
import glob
import time

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras import Input
from keras.applications import VGG19
from keras.callbacks import TensorBoard
from keras.layers import BatchNormalization, Activation, LeakyReLU, Add, Dense
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.models import Model
from keras.optimizers import Adam
from scipy.misc import imread, imresize
import keras.backend as K
import cv2
import os
from PIL import Image
# from imageio import imread
# from skimage.transform import resize

### Residual Block

In [8]:
def residual_block(x):
    """
    Residual block
    """
    filters = [64, 64]
    kernel_size = 3
    strides = 1
    padding = "same"
    momentum = 0.8
    activation = "relu"

    res = Conv2D(filters=filters[0], kernel_size=kernel_size, strides=strides, padding=padding)(x)
    res = Activation(activation=activation)(res)
    res = BatchNormalization(momentum=momentum)(res)

    res = Conv2D(filters=filters[1], kernel_size=kernel_size, strides=strides, padding=padding)(res)
    res = BatchNormalization(momentum=momentum)(res)

    # Add res and x
    res = Add()([res, x])
    return res

### Build Generator

In [9]:
def build_generator():
    """
    Create a generator network using the hyperparameter values defined below
    :return:
    """
    residual_blocks = 16
    momentum = 0.8
    input_shape = (64, 64, 3)

    # Input Layer of the generator network
    input_layer = Input(shape=input_shape)

    # Add the pre-residual block
    gen1 = Conv2D(filters=64, kernel_size=9, strides=1, padding='same', activation='relu')(input_layer)

    # Add 16 residual blocks
    res = residual_block(gen1)
    for i in range(residual_blocks - 1):
        res = residual_block(res)

    # Add the post-residual block
    gen2 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(res)
    gen2 = BatchNormalization(momentum=momentum)(gen2)

    # Take the sum of the output from the pre-residual block(gen1) and the post-residual block(gen2)
    gen3 = Add()([gen2, gen1])

    # Add an upsampling block
    gen4 = UpSampling2D(size=2)(gen3)
    gen4 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(gen4)
    gen4 = Activation('relu')(gen4)

    # Add another upsampling block
    gen5 = UpSampling2D(size=2)(gen4)
    gen5 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(gen5)
    gen5 = Activation('relu')(gen5)

    # Output convolution layer
    gen6 = Conv2D(filters=3, kernel_size=9, strides=1, padding='same')(gen5)
    output = Activation('tanh')(gen6)

    # Keras model
    model = Model(inputs=[input_layer], outputs=[output], name='generator')
    return model


### Build Descriminator

In [10]:
def build_discriminator():
    """
    Create a discriminator network using the hyperparameter values defined below
    :return:
    """
    leakyrelu_alpha = 0.2
    momentum = 0.8
    input_shape = (256, 256, 3)

    input_layer = Input(shape=input_shape)

    # Add the first convolution block
    dis1 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(input_layer)
    dis1 = LeakyReLU(alpha=leakyrelu_alpha)(dis1)

    # Add the 2nd convolution block
    dis2 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(dis1)
    dis2 = LeakyReLU(alpha=leakyrelu_alpha)(dis2)
    dis2 = BatchNormalization(momentum=momentum)(dis2)

    # Add the third convolution block
    dis3 = Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(dis2)
    dis3 = LeakyReLU(alpha=leakyrelu_alpha)(dis3)
    dis3 = BatchNormalization(momentum=momentum)(dis3)

    # Add the fourth convolution block
    dis4 = Conv2D(filters=128, kernel_size=3, strides=2, padding='same')(dis3)
    dis4 = LeakyReLU(alpha=leakyrelu_alpha)(dis4)
    dis4 = BatchNormalization(momentum=0.8)(dis4)

    # Add the fifth convolution block
    dis5 = Conv2D(256, kernel_size=3, strides=1, padding='same')(dis4)
    dis5 = LeakyReLU(alpha=leakyrelu_alpha)(dis5)
    dis5 = BatchNormalization(momentum=momentum)(dis5)

    # Add the sixth convolution block
    dis6 = Conv2D(filters=256, kernel_size=3, strides=2, padding='same')(dis5)
    dis6 = LeakyReLU(alpha=leakyrelu_alpha)(dis6)
    dis6 = BatchNormalization(momentum=momentum)(dis6)

    # Add the seventh convolution block
    dis7 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(dis6)
    dis7 = LeakyReLU(alpha=leakyrelu_alpha)(dis7)
    dis7 = BatchNormalization(momentum=momentum)(dis7)

    # Add the eight convolution block
    dis8 = Conv2D(filters=512, kernel_size=3, strides=2, padding='same')(dis7)
    dis8 = LeakyReLU(alpha=leakyrelu_alpha)(dis8)
    dis8 = BatchNormalization(momentum=momentum)(dis8)

    # Add a dense layer
    dis9 = Dense(units=1024)(dis8)
    dis9 = LeakyReLU(alpha=0.2)(dis9)

    # Last dense layer - for classification
    output = Dense(units=1, activation='sigmoid')(dis9)

    model = Model(inputs=[input_layer], outputs=[output], name='discriminator')
    return model

### Build VGG19

In [11]:
def build_vgg():
    """
    Build VGG network to extract image features
    """
    input_shape = (256, 256, 3)

    # Load a pre-trained VGG19 model trained on 'Imagenet' dataset
    vgg = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
    vgg.outputs = [vgg.layers[20].output]

    # Create a Keras model
    model = Model(vgg.input, vgg.outputs)
    return model


# def build_vgg():
#     """
#     Build VGG network to extract image features
#     """
#     input_shape = (256, 256, 3)

#     # Load a pre-trained VGG19 model trained on 'Imagenet' dataset
#     vgg = VGG19(include_top=False, weights='imagenet')
#     vgg.outputs = [vgg.layers[20].output]

#     input_layer = Input(shape=input_shape)

#     # Extract features
#     features = vgg(input_layer)

#     # Create a Keras model
#     model = Model(inputs=[input_layer], outputs=[features])
#     return model

In [12]:
model = build_vgg()
model.summary()

80142336/80134624 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
______________________________________________________________

### Sample Images

In [13]:
def sample_images(data_dir, batch_size, high_resolution_shape, low_resolution_shape):
    # Make a list of all images inside the data directory
    all_images = glob.glob(data_dir)

    # Choose a random batch of images
    images_batch = np.random.choice(all_images, size=batch_size)

    low_resolution_images = []
    high_resolution_images = []

    for img in images_batch:
        # Get an ndarray of the current image
        img1 = imread(img, mode='RGB')
        img1 = img1.astype(np.float32)

        # Resize the image
        img1_high_resolution = imresize(img1, high_resolution_shape)
        img1_low_resolution = imresize(img1, low_resolution_shape)

        # Do a random horizontal flip
        if np.random.random() < 0.5:
            img1_high_resolution = np.fliplr(img1_high_resolution)
            img1_low_resolution = np.fliplr(img1_low_resolution)

        high_resolution_images.append(img1_high_resolution)
        low_resolution_images.append(img1_low_resolution)

    # Convert the lists to Numpy NDArrays
    return np.array(high_resolution_images), np.array(low_resolution_images)

### Save Images

In [14]:
def psnr(y_true,y_pred):
    return -10*K.log(K.mean(K.flatten((y_true-y_pred))**2))/np.log(10)

# Loss functtion
def ssim_loss(y_true, y_pred):
  return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0))

In [18]:
def save_images(low_resolution_image, original_image, generated_image, path):
    """
    Save low-resolution, high-resolution(original) and
    generated high-resolution images in a single image
    """
    fig = plt.figure(figsize=(12,5))
    ax = fig.add_subplot(1, 3, 1)
    ax.imshow(low_resolution_image)
    ax.axis("off")
    ax.set_title("Low-resolution")

    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(original_image)
    ax.axis("off")
    ax.set_title(f"High-resolution")
    # ax.save(hr_path,bbox_inches='tight',transparent=True, pad_inches=0)

    ax = fig.add_subplot(1, 3, 3)
    ax.imshow(np.squeeze(generated_image), cmap = plt.get_cmap(name = 'gray'))
    ax.axis("off")
    ax.set_title(f"Generated")
    # ax.save(pr_path, bbox_inches='tight',transparent=True, pad_inches=0)

    plt.savefig(path)


# def save_images(low_resolution_image, original_image, generated_image, path):
#     """
#     Save low-resolution, high-resolution(original) and
#     generated high-resolution images in a single image
#     """
#     plt.figure(figsize=(14, 7))
#     ax = plt.subplot(1, 3, 3)
#     generated_image = cv2.cvtColor(generated_image, cv2.COLOR_BGR2GRAY)
#     plt.imshow(generated_image)
#     plt.xlabel(f"SSIM LOSS : {ssim_loss(generated_image, original_image)}")
#     plt.title("Generated Image")

#     ax = plt.subplot(1, 3, 2)
#     plt.imshow(original_image)
#     plt.xlabel(f"PSNR : {psnr(generated_image, original_image)}")
#     plt.title("High Resolution Image")

#     ax = plt.subplot(1, 3, 1)
#     plt.imshow(np.squeeze(low_resolution_image))
#     plt.title("Low Resoluion Image")
#     plt.xlabel(f"{name}")

#     plt.savefig(path, dpi=100, pad_inches=0.0, bbox_inches='tight')

### Write a Log

In [19]:
# !pip install sewar

### Final SRGAN Execution

In [20]:
from sewar.full_ref import *

ModuleNotFoundError: ignored

In [ ]:
if __name__ == '__main__':
    data_dir = "/content/train/*.*"
    os.makedirs("results", exist_ok=True)
    # os.makedirs("HR", exist_ok=True)
    # os.makedirs("PR", exist_ok=True)

    epochs = 30000
    batch_size = 1
    mode = 'train'

    # Shape of low-resolution and high-resolution images
    low_resolution_shape = (64, 64, 3)
    high_resolution_shape = (256, 256, 3)

    # Common optimizer for all networks
    common_optimizer = Adam(0.0002, 0.5)


    if mode == 'train':
        # Build and compile VGG19 network to extract features
        vgg = build_vgg()
        vgg.trainable = False
        vgg.compile(loss='mse', optimizer=common_optimizer, metrics=['accuracy', psnr, ssim_loss])

        # Build and compile the discriminator network
        discriminator = build_discriminator()
        discriminator.compile(loss='mse', optimizer=common_optimizer, metrics=['accuracy', psnr, ssim_loss])

        # Build the generator network
        generator = build_generator()

        """
        Build and compile the adversarial model
        """

        # Input layers for high-resolution and low-resolution images
        input_high_resolution = Input(shape=high_resolution_shape)
        input_low_resolution = Input(shape=low_resolution_shape)

        # Generate high-resolution images from low-resolution images
        generated_high_resolution_images = generator(input_low_resolution)

        # Extract feature maps of the generated images
        features = vgg(generated_high_resolution_images)

        # Make the discriminator network as non-trainable
        discriminator.trainable = False

        # Get the probability of generated high-resolution images
        probs = discriminator(generated_high_resolution_images)

        # Create and compile an adversarial model
        adversarial_model = Model([input_low_resolution, input_high_resolution], [probs, features])
        adversarial_model.compile(loss=['binary_crossentropy', 'mse'], loss_weights=[1e-3, 1], optimizer=common_optimizer)

        # Add Tensorboard
        tensorboard = TensorBoard(log_dir="logs/".format(time.time()))
        tensorboard.set_model(generator)
        tensorboard.set_model(discriminator)

        for epoch in range(epochs):
            print("Epoch:{}".format(epoch))

            """
            Train the discriminator network
            """

            # Sample a batch of images
            high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                          low_resolution_shape=low_resolution_shape,
                                                                          high_resolution_shape=high_resolution_shape)
            # Normalize images
            high_resolution_images = high_resolution_images / 127.5 - 1.
            low_resolution_images = low_resolution_images / 127.5 - 1.

            # Generate high-resolution images from low-resolution images
            generated_high_resolution_images = generator.predict(low_resolution_images)

            # Generate batch of real and fake labels
            real_labels = np.ones((batch_size, 16, 16, 1))
            fake_labels = np.zeros((batch_size, 16, 16, 1))

            # Train the discriminator network on real and fake images
            d_loss_real = discriminator.train_on_batch(high_resolution_images, real_labels)
            d_loss_fake = discriminator.train_on_batch(generated_high_resolution_images, fake_labels)

            # Calculate total discriminator loss
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            print("d_loss:", d_loss)

            """
            Train the generator network
            """

            # Sample a batch of images
            high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                          low_resolution_shape=low_resolution_shape,
                                                                          high_resolution_shape=high_resolution_shape)
            # Normalize images
            high_resolution_images = high_resolution_images / 127.5 - 1.
            low_resolution_images = low_resolution_images / 127.5 - 1.

            # Extract feature maps for real high-resolution images
            image_features = vgg.predict(high_resolution_images)

            # Train the generator network
            g_loss = adversarial_model.train_on_batch([low_resolution_images, high_resolution_images], [real_labels, image_features])

            print("g_loss:", g_loss)

            # Write the losses to Tensorboard
            # write_log(tensorboard, 'g_loss', g_loss[0], epoch)
            # write_log(tensorboard, 'd_loss', d_loss[0], epoch)

            # Sample and save images after every 100 epochs
            if epoch % 100 == 0:
                high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                              low_resolution_shape=low_resolution_shape,
                                                                              high_resolution_shape=high_resolution_shape)
                # Normalize images
                high_resolution_images = high_resolution_images / 127.5 - 1.
                low_resolution_images = low_resolution_images / 127.5 - 1.

                generated_images = generator.predict_on_batch(low_resolution_images)
                # print(generated_images.shape, high_resolution_images.shape)

                for index, img in enumerate(generated_images):
                    img = np.mean(img, axis=2)
                    save_images(low_resolution_images[index], high_resolution_images[index], img, path="/content/results/img_{}_{}".format(epoch, index))

        # Save models
        generator.save_weights("generator.h5")
        discriminator.save_weights("discriminator.h5")

    if mode == 'predict':
        # Build and compile the discriminator network
        discriminator = build_discriminator()

        # Build the generator network
        generator = build_generator()

        # Load models
        generator.load_weights("generator.h5")
        discriminator.load_weights("discriminator.h5")

        # Get 10 random images
        high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=10,
                                                                      low_resolution_shape=low_resolution_shape,
                                                                      high_resolution_shape=high_resolution_shape)
        # Normalize images
        high_resolution_images = high_resolution_images / 127.5 - 1.
        low_resolution_images = low_resolution_images / 127.5 - 1.

        # Generate high-resolution images from low-resolution images
        generated_images = generator.predict_on_batch(low_resolution_images)
        # generated_images = cv2.cvtColor(generated_images, cv2.COLOR_BGR2GRAY)
        
        # Save images
        for index, img in enumerate(generated_images):
            img = np.mean(img, axis=2)
            save_images(low_resolution_images[index], high_resolution_images[index], img, path="/content/results/gen_{}".format(index))

Epoch:0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


d_loss: [0.31398254 0.49804688 5.51269507 0.04274057]
g_loss: [0.08572763949632645, 0.6055980324745178, 0.08512204140424728]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch:1
d_loss: [0.25454995 0.56640625 6.66638684 0.03252118]
g_loss: [0.08682255446910858, 0.946404218673706, 0.08587615191936493]
Epoch:2
d_loss: [0.22229281 0.59960938 6.70447206 0.16874399]
g_loss: [0.06673730909824371, 0.6492639183998108, 0.06608804315328598]
Epoch:3
d_loss: [0.21388377 0.65234375 6.70431352 0.02101163]
g_loss: [0.05744236335158348, 1.6266735792160034, 0.055815689265728]
Epoch:4
d_loss: [ 0.11451178  0.81640625 10.52445936  0.22584346]
g_loss: [0.05402672290802002, 3.2902727127075195, 0.0507364496588707]
Epoch:5
d_loss: [ 0.02764062  0.9765625  16.20135641  0.31599618]
g_loss: [0.04579101875424385, 5.1970109939575195, 0.040594007819890976]
Epoch:6
d_loss: [ 0.02142906  0.984375   17.89403343  0.18250406]
g_loss: [0.047919631004333496, 5.526231288909912, 0.04239340126514435]
Epoch:7
d_loss: [ 0.02845037  0.9921875  17.01652575  0.1413588 ]
g_loss: [0.04921683669090271, 0.036548636853694916, 0.0491802878677845]
Epoch:8
d_loss: [5.71848406e-03 1.00000000e+00 2.258562

...

g_loss: [0.0028408917132765055, 0.1711362898349762, 0.00266975536942482]
Epoch:6801
d_loss: [2.93820157e-09 1.00000000e+00 8.53265724e+01 9.99996930e-01]
g_loss: [0.0029563601128757, 0.2302013635635376, 0.0027261588256806135]
Epoch:6802
d_loss: [4.85376517e-09 1.00000000e+00 8.32358742e+01 9.99994606e-01]
g_loss: [0.0038170386105775833, 0.3171355426311493, 0.003499903017655015]
Epoch:6803
d_loss: [3.83457688e-09 1.00000000e+00 8.41879387e+01 9.99995291e-01]
g_loss: [0.003308790037408471, 0.23146574199199677, 0.0030773242469877005]
Epoch:6804
d_loss: [3.21509097e-09 1.00000000e+00 8.49311218e+01 9.99998152e-01]
g_loss: [0.003335776971653104, 0.31124627590179443, 0.0030245305970311165]
Epoch:6805
d_loss: [4.06838452e-09 1.00000000e+00 8.39194336e+01 9.99992162e-01]
g_loss: [0.003488813526928425, 0.22626963257789612, 0.0032625438179820776]
Epoch:6806
d_loss: [2.73881817e-09 1.00000000e+00 8.57214813e+01 9.99998152e-01]
g_loss: [0.003801756538450718, 0.1556437611579895, 0.00364611274562776

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


g_loss: [0.002557791769504547, 0.06863425672054291, 0.002489157486706972]
Epoch:6901
d_loss: [3.01045588e-09 1.00000000e+00 8.53095703e+01 9.99998629e-01]
g_loss: [0.0032349370885640383, 0.07180068641901016, 0.0031631363090127707]
Epoch:6902
d_loss: [6.10213013e-09 1.00000000e+00 8.21460075e+01 9.99996811e-01]
g_loss: [0.003458370454609394, 0.05728732421994209, 0.0034010831732302904]
Epoch:6903
d_loss: [7.16655402e-09 1.00000000e+00 8.18122902e+01 9.99997675e-01]
g_loss: [0.002766714198514819, 0.08913372457027435, 0.002677580574527383]
Epoch:6904
d_loss: [6.40777276e-09 1.00000000e+00 8.20572815e+01 9.99996930e-01]
g_loss: [0.003221366088837385, 0.17768704891204834, 0.003043679054826498]
Epoch:6905
d_loss: [7.94679589e-09 1.00000000e+00 8.10079575e+01 9.99998033e-01]
g_loss: [0.002417734358459711, 0.09730036556720734, 0.0023204339668154716]
Epoch:6906
d_loss: [1.15616183e-08 1.00000000e+00 7.97441139e+01 9.99997973e-01]
g_loss: [0.0027140318416059017, 0.09532449394464493, 0.00261870725

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


g_loss: [0.0035365717485547066, 0.1879802942276001, 0.0033485915046185255]
Epoch:7001
d_loss: [3.38334638e-09 1.00000000e+00 8.48001175e+01 9.99998629e-01]
g_loss: [0.003662609960883856, 0.31624072790145874, 0.0033463691361248493]
Epoch:7002
d_loss: [3.53995344e-09 1.00000000e+00 8.45817337e+01 9.99995917e-01]
g_loss: [0.0034805757459253073, 0.17658893764019012, 0.0033039867412298918]
Epoch:7003
d_loss: [2.60917210e-09 1.00000000e+00 8.58355446e+01 9.99996394e-01]
g_loss: [0.003469982650130987, 0.2393738031387329, 0.003230608766898513]
Epoch:7004
d_loss: [2.98781822e-09 1.00000000e+00 8.53040810e+01 9.99998927e-01]
g_loss: [0.003235897049307823, 0.24046526849269867, 0.002995431888848543]
Epoch:7005
d_loss: [1.45326029e-08 1.00000000e+00 8.00601425e+01 9.99998450e-01]
g_loss: [0.003614980960264802, 0.16503562033176422, 0.003449945244938135]
Epoch:7006
d_loss: [1.82175875e-08 1.00000000e+00 7.96622543e+01 9.99993116e-01]
g_loss: [0.004156970884650946, 0.10120180994272232, 0.0040557691827

In [ ]:
!rm -rf /content/results/*